In [ ]:
from dolfinx import *
from dolfinx import default_scalar_type
from dolfinx.fem import *
from mpi4py import *
from dolfinx.fem.petsc import LinearProblem
from dolfinx.plot import vtk_mesh
from petsc4py import PETSc
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import math
import pyvista
import ufl
import basix

In [ ]:
def t_0(x):
    return 1.-x[1] + 0.2*np.cos(x[0]*np.pi)*np.sin(x[1]*np.pi)

In [ ]:
# code for Stokes Equation
ne = 10
p = 1
eta = 1

# define the domain (unit square) of length and width ne, running in parallel
domain = mesh.create_unit_square(MPI.COMM_WORLD, ne, ne)

# define extra variables
Ra = Constant(domain, default_scalar_type(10000.0))
gravity = Constant(domain, default_scalar_type((0.0,-1.0)))

# define velocity and pressure element (and temperature element)
v_e = basix.ufl.element("Lagrange", domain.basix_cell(), p+1, shape=(domain.geometry.dim,))
p_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)
t_e = basix.ufl.element("Lagrange", domain.basix_cell(), p)

# define mixed element
vPe = basix.ufl.mixed_element([v_e, p_e])

# define function spaces (mixed element and temperature)
V = functionspace(domain, vPe)
VT = functionspace(domain, t_e)
T_i = Function(VT)
T_i.interpolate(t_0)
vp_i = Function(V)
v_i = vp_i.sub(0)

# define Velocity and Pressure function space
V_v, _ = V.sub(0).collapse()
V_vx, _ = V_v.sub(0).collapse()
V_vy, _ = V_v.sub(1).collapse()
V_p, _ = V.sub(1).collapse()

# define test functions
v_t, p_t = ufl.TestFunctions(V)
t_t = ufl.TestFunction(VT)

# define trial functions
trial_v, trial_p = ufl.TrialFunctions(V)
trial_t = ufl.TrialFunction(VT)

# define boundary conditions
# x = 0 or x = 1 (left and right boundaries)
def boundary_D_1(x):
    return np.logical_or(np.isclose(x[0], 0), np.isclose(x[0], 1))

dofs_D_1 = locate_dofs_geometrical((V.sub(0).sub(0), V_vx), boundary_D_1)
u_bc1 = Function(V_vx)
u_bc1.x.array[:] = 0
bc1 = dirichletbc(u_bc1, dofs_D_1, V.sub(0).sub(0))

# y = 0 or y = 1 (top and bottom boundaries)
def boundary_D_2(x):
    return np.logical_or(np.isclose(x[1], 0), np.isclose(x[1], 1))

dofs_D_2 = locate_dofs_geometrical((V.sub(0).sub(1), V_vy), boundary_D_2)
u_bc2 = Function(V_vy)
u_bc2.x.array[:] = 0
bc2 = dirichletbc(u_bc2, dofs_D_2, V.sub(0).sub(1))

# p = 0 (Pressure at origin)
def boundary_D_3(x):
    return np.logical_and(np.isclose(x[0], 0), np.isclose(x[1], 0))

dofs_D_3 = locate_dofs_geometrical((V.sub(1), V_p), boundary_D_3)
u_bc3 = Function(V_p)
u_bc3.x.array[:] = 0
bc3 = dirichletbc(u_bc3, dofs_D_3, V.sub(1))

# assemble weak form
K = ufl.inner(ufl.sym(ufl.grad(v_t)), 2*eta*ufl.sym(ufl.grad(trial_v)))*ufl.dx
G = -ufl.div(v_t)*trial_p*ufl.dx
D = -p_t*ufl.div(trial_v)*ufl.dx
S = K + G + D

L = -ufl.inner(v_t, gravity)*Ra*T_i*ufl.dx

# assemble problem solver and solve
problem = LinearProblem(S, L, bcs=[bc1, bc2, bc3], u=vp_i, petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
vp_i = problem.solve()

In [ ]:
# code for Heat equation

# define temperature boundary conditions
def boundary_D_4(x):
    return np.isclose(x[1], 1)

dofs_D_4 = locate_dofs_geometrical(VT, boundary_D_4)
u_bc4 = Function(VT)
u_bc4.x.array[:] = 0
bc4 = dirichletbc(u_bc4, dofs_D_4)

def boundary_D_5(x):
    return np.isclose(x[1], 0)
    
dofs_D_5 = locate_dofs_geometrical(VT, boundary_D_5)
u_bc5 = Function(VT)
u_bc5.x.array[:] = 1
bc5 = dirichletbc(u_bc5, dofs_D_5)

# assemble weak form
ST = (t_t*ufl.inner(v_i, ufl.grad(trial_t)) + ufl.inner(ufl.grad(t_t), ufl.grad(trial_t)))*ufl.dx

zeroes = Constant(domain, default_scalar_type(0.0))
LT = zeroes*t_t*ufl.dx

problemT = LinearProblem(ST, LT, bcs=[bc4, bc5], u=T_i, petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
T_i = problemT.solve()

In [ ]:
def visualize(vh):
    pyvista.start_xvfb()

    pyvista_cells, cell_types, geometry = vtk_mesh(vh.function_space)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)

    values = np.zeros((geometry.shape[0], 3))
    values[:, :len(vh)] = vh.x.array.real.reshape((geometry.shape[0], len(vh)))
    grid["u"] = values
    geom = pyvista.Arrow()
    glyphs = grid.glyph(orient="u", geom=geom, factor=0.007)

    plotter = pyvista.Plotter(window_size=[800,800])
    plotter.add_mesh(grid, show_edges=True, show_scalar_bar=False, color='w')
    plotter.add_mesh(glyphs, cmap='coolwarm', show_scalar_bar=True)
    plotter.add_text("Blankenbach Convection", position="upper_edge", font_size=14, color="black")
    plotter.view_xy()
    plotter.show()

In [ ]:
def visualize2(th):
    pyvista.start_xvfb()

    pyvista_cells, cell_types, geometry = vtk_mesh(th.function_space)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
    grid.point_data["u"] = th.x.array
    grid.set_active_scalars("u")

    plotter = pyvista.Plotter()
    plotter.add_text("Temperature Solution", position="upper_edge", font_size=14, color="black")
    plotter.add_mesh(grid, show_edges=True)
    plotter.view_xy()
    
    if not pyvista.OFF_SCREEN:
        plotter.show()
    else:
        figure = plotter.screenshot("blankenbachTemp.png")

In [ ]:
visualize(v_i.collapse())
visualize2(T_i)

n = 0
while n < 10:
    n+=1